In [48]:
import yfinance as yf
import pandas as pd

#Solana (SOL-USD)
dados = yf.download('SOL-USD', start='2020-04-10', end='2024-10-21', interval='1d')
dados.reset_index(inplace=True)

dados = dados[['Date', 'Close']]

dados.tail()


[*********************100%***********************]  1 of 1 completed


Price,Date,Close
Ticker,,SOL-USD
1650,2024-10-16 00:00:00+00:00,154.234222
1651,2024-10-17 00:00:00+00:00,150.316025
1652,2024-10-18 00:00:00+00:00,154.975937
1653,2024-10-19 00:00:00+00:00,159.639359
1654,2024-10-20 00:00:00+00:00,166.939819
